# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга— специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. 

### План работы:
1. Изучить информацию о входных данных: просмотреть информацию о содержании таблицы, наметить, какие данные требуют корректировки.
2. Выявить и обработать пропуски.
3. Заменить тип данных.
4. Обработать дубликаты.
5. Провести лемматизацию.
6. Провести категоризацию.
7. Выяснить ответы на вопросы, являющиеся ключевой целью исследования:
  * Есть ли зависимость между наличием детей и возвратом кредита в срок?
  * Есть ли зависимость между семейным положением и возвратом кредита в срок?
  * Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
  * Как разные цели кредита влияют на его возврат в срок?
8. Сформулировать общие выводы по исследованию.

### Загрузка файла с данными, изучение общей информации 

In [1]:
import pandas as pd #импортируем библиотеки
from pymystem3 import Mystem 
m = Mystem()

In [2]:
borrowers = pd.read_csv('/datasets/data.csv') #прочитаем файл
borrowers.to_csv('credits_dataset')

In [3]:
borrowers.info() #посмотрим общую информацию о таблице
borrowers.head(5) #выведем первые 5 строк

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Таблица содержит столбцы:
* children - количество детей в семье
* days_employed - общий трудовой стаж в днях
* dob_years - возраст клиента в годах
* education - уровень образования клиента
* education_id - идентификатор уровня образования
* family_status - семейное положение
* family_status_id - идентификатор семейного положения
* gender - пол клиента
* income_type - тип занятости
* debt - имел ли задолженность по возврату кредитов
* total_income - ежемесячный доход
* purpose - цель получения кредита
  

В результате первичного просмотра информации видно, что столбцы days_employed и total_income имеют тип float64. Для удобства и экономии памяти их можно будет заменить на int.  
В столбце days_employed видим много отрицательных значений, с которыми нужно будет работать.   
В столбце education буквы напечатаны в разных регистрах.  
Столбец  purpose перечисляет цели, во многом подобные друг другу. Нужно их привести к более компактному виду.   

###  Предобработка данных

#### ***Обработка пропусков***####

In [4]:
borrowers.isnull().sum() #ищем пропуски

borrowers['income_type'].value_counts() #смотрим количество значений по типам занятости

types_income = borrowers['income_type'].unique() #создаем переменную с уникальными значениями
for i in types_income: #пишем цикл для заполнения пропусков медианой
    median = borrowers[borrowers['income_type'] == i]['total_income'].median()
    borrowers.loc[borrowers['income_type'] == i, 'total_income'] = borrowers.loc\
    [borrowers['income_type'] == i, 'total_income'].fillna(median)
    
borrowers.isnull().sum() #проверяем результат

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Столбец days_employed имеет слишком много отрицательных значений, а положительные далеки от реальности (трудовой стаж слишком большой для обычного человека. Для решения задачи, поставленной заказчиком, этот столбец не требуется, на результат не влияет, поэтому его можно удалить из таблицы.

Определяем пропущенные значения.  
Значения пропущены в столбце "total_income"(ежемесячный доход). Возможно, причиной тому техническая ошибка, поскольку обычно этому пункту уделяется большое внимание при оформлении кредита. Поскольку это важная информация, то не следует удалять соответствующие строки.  
Это количественные переменные, поэтому пропуски будем заполнять характерными значениями, находя медиану, фильтруя по типу занятости.  
Создаем цикл, который находит медиану ежемесячного дохода и заполняет пропущенные значения в ежемесячном доходе по категориям соответствующими медианными значениями.

#### ***Замена типа данных***####

In [5]:
borrowers['total_income'] = borrowers['total_income'].astype(int) #заменяем тип данных
borrowers.info()
borrowers['children'].value_counts() #смотрим количество значений по столбцу

borrowers['children'].median() #находим медиану

borrowers.loc[borrowers['children'] == 20, 'children'] = 0 #заменяем некорректные значения медианой
borrowers.loc[borrowers['children'] == -1, 'children'] = 0
borrowers['children'].value_counts() #проверяем результат

#приводим столбец к единому нижнему регистру
borrowers['education'] = borrowers['education'].str.lower() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Столбец total_income требует замены вещественного типа данных на целочисленный. Заменяем методом astype(), поскольку пропуски уже были заменены ранее.  
*Существует также возможность замены типа данных методом to_numeric(), с аргументами downcast = 'integer' и errors='coerce', но метод astype() в данном случае кажется предпочтительнее.*

Также присутствует необходимость заменить некоторые другие данные.  
При просмотре столбца children выявлены ошибки.  
Число детей не может быть равно -1, а также маловероятно, что у 76 человек из базы число детей равно 20.  
вычислим медиану и заменим этим значением ошибочные данные.  
В столбце education различаются регистры названий. Приведем их к единому регистру.

#### ***Обработка дубликатов***####

In [6]:
borrowers.duplicated().sum()  #выводим количество дубликатов

borrowers = borrowers.drop_duplicates().reset_index(drop=True) #удаляем

Выявляем дубликаты: их 54.  
Возможно, они появились в результате ошибки при внесении данных в систему.  
Для их поиска используем метод duplicated().sum(), для удаления - метод drop_duplicates().

#### ***Лемматизация***####

In [7]:
#пишем цикл, в котором проводится лемматизация значений столбца, замена общих значений на ключевые
#и добавление результатов в новый столбец
def change_purpose(purpose_string): 
    purpose_list = m.lemmatize(purpose_string)
    for name in purpose_list:
        if name == 'жилье':
            purpose_string = 'недвижимость'
            return purpose_string
        if name == 'недвижимость':
            purpose_string = 'недвижимость'
            return purpose_string
        if name == 'образование':
            purpose_string = 'образование'
            return purpose_string
        if name == 'свадьба':
            purpose_string = 'свадьба'
            return purpose_string
        if name == 'автомобиль':
            purpose_string = 'автомобиль'
            return purpose_string
borrowers['purpose_lem'] = borrowers['purpose'].apply(change_purpose)
borrowers.head(5) #выводим первые 5 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Нужно лемматизировать столбец с целями получения кредита.  
Лемматизируем исходную строку.  
Напишем цикл, заменяющий строку на ключевое слово, когда оно в ней есть.  
Присоединим результаты выполнения цикла к таблице в качестве нового столбца purpose_lem.

#### ***Категоризация данных***####

In [8]:
pd.options.display.float_format = '{:.0f}'.format
print(borrowers.describe()) #смотрим срезы таблицы по значениям

#напишем цикл, который будет возвращать класс уровня доходов по возрастанию от 1 до 4
def total_income_group(total_income): 
       
        if total_income <= 107798:
                return 'категория 1'
        if total_income <= 142594:
                return 'категория 2'
        if total_income <= 195549:
                return 'категория 3'
        return 'категория 4'
#создадим отдельный столбец с категориями заработка и в его ячейках запишем значения, 
#возвращаемые функцией.
borrowers['income_group'] = borrowers['total_income'].apply(total_income_group)
borrowers.head(10)

       children  days_employed  dob_years  education_id  family_status_id  \
count     21454          19351      21454         21454             21454   
mean          0          63046         43             1                 1   
std           1         140827         13             1                 1   
min           0         -18389          0             0                 0   
25%           0          -2747         33             1                 0   
50%           0          -1203         42             1                 0   
75%           1           -291         53             1                 1   
max           5         401755         75             4                 4   

       debt  total_income  
count 21454         21454  
mean      0        165320  
std       0         98187  
min       0         20667  
25%       0        107623  
50%       0        142594  
75%       0        195820  
max       1       2265604  


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lem,income_group
0,1,-8438,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,категория 4
1,1,-4025,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,категория 2
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,категория 3
3,3,-4125,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,категория 4
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,категория 3
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,категория 4
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,категория 4
7,0,-153,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,категория 2
8,2,-6930,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,категория 1
9,0,-2189,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,категория 3


Вызовем метод describe() для нахождения верхнего, нижнего и среднего квартилей в категориях. Отсюда видно, что минимальное пороговое значение дохода (у 25% заемщиков) 107798р, следующий срез 25% заемщиков получает до 142594р, еще 25% имеют доход до 195550р и оставшиеся 25% получают свыше 2.265 млн.р. 

Напишем цикл, который будет возвращать класс уровня доходов по возрастанию от 1 до 4, где 1 - самый низкий уровень дохода, 4 - самый высокий.  
Создадим отдельный столбец с категориями заработка и в его ячейках запишем значения, возвращаемые функцией.

### ***Есть ли зависимость между наличием детей и возвратом кредита в срок?***

In [9]:
pd.options.display.float_format = '{:.3f}'.format

#создаем сводную таблицу с количеством детей и задолженностями
pivot_children = borrowers.pivot_table(index = 'children', values = 'debt',\
                                       aggfunc = ['sum', 'count', 'mean'])

#создаем сводную таблицу с отображением соотношения должников в процентах
pivot_children = borrowers.pivot_table(index = ['children'], values = 'debt')
pivot_children['debt'] = pivot_children['debt'].apply(lambda x: '{:.2%}'.format(x))
pivot_children

,debt
children,
0,7.54%
1,9.23%
2,9.45%
3,8.18%
4,9.76%
5,0.00%


Создадим сводную таблицу, где выведем количество детей и количество клиентов: общее и с задолженностью. Вычислим процентное отношение этих величин при разном количестве детей.  
В результате видим, что наивысший процент задолженностей имеют клиенты с одним, двумя и четырьмя детьми, хотя и имеющие троих детей находятся примерно на том же уровне. Данные о имеющих пятерых детей несущественны для статистики, поскольку таких случаев слишком мало.  
Таким образом, меньше всего процент задолженностей у бездетных.

### ***Есть ли зависимость между семейным положением и возвратом кредита в срок?***###

In [10]:
#создаем сводную таблицу с семейным положением и задолженностями
pivot_family_status = borrowers.pivot_table(index = 'family_status', values = 'debt',\
                                            aggfunc = ['sum', 'count', 'mean'])

#создаем сводную таблицу с отображением соотношения должников в процентах
pivot_family_status = borrowers.pivot_table(index = ['family_status'], values = 'debt')
pivot_family_status['debt'] = pivot_family_status['debt'].apply(lambda x: '{:.2%}'.format(x))
pivot_family_status

,debt
family_status,
Не женат / не замужем,9.75%
в разводе,7.11%
вдовец / вдова,6.57%
гражданский брак,9.35%
женат / замужем,7.55%


Создадим сводную таблицу, где выведем все типы семейного статуса и количество клиентов: общее и с задолженностью. Вычислим процентное отношение этих величин при разном семейном статусе.  
В результате видим, что наивысший процент задолженностей имеют клиенты, находящиеся не в браке либо живущие в гражданском браке. Чуть лучше обстоит дело с возвратом задолженностей  у состоящих в браке и разведенных. Меньше всего процент задолженности у вдовцов.

### ***Есть ли зависимость между уровнем дохода и возвратом кредита в срок?***###

In [11]:
#выводим общее количество клиентов по категориям заработка, найденным ранее с помощью категоризации
pivot_income_group = borrowers.pivot_table(index = 'income_group', values = 'debt',\
                                           aggfunc = ['sum', 'count', 'mean'])


#создаем сводную таблицу с отображением соотношения должников в процентах
pivot_income_group = borrowers.pivot_table(index = ['income_group'], values = 'debt')
pivot_income_group['debt'] = pivot_income_group['debt'].apply(lambda x: '{:.2%}'.format(x))
pivot_income_group

,debt
income_group,
категория 1,7.93%
категория 2,8.84%
категория 3,8.51%
категория 4,7.17%


Создадим сводную таблицу, где выведем количество клиентов по категориям заработка, найденным ранее с помощью категоризации: общее и с задолженностью. Вычислим процентное отношение этих величин в разных группах доходов.  
В результате видим, что наивысший процент задолженностей имеют клиенты, находящиеся в средних группах по уровню зарплаты. Для категории 2 задолженность имеют 385 человек из 4419 или 8.71%. Для 3 категории - 543 человека из 6343 или 8.56%. Чуть лучше обстоят дела с возвратом кредита у самой низшей, 1 категории (427 человек из 5381 или 7.94%). И лучше всего ситуация у 4 категории (долг имеют 386 человек из 5382 или 7.17%).

### ***Как разные цели кредита влияют на его возврат в срок?***###

In [12]:
#создаем сводную таблицу с целями и задолженностями
pivot_purpose_debt = borrowers.pivot_table(index = 'purpose_lem', values = 'debt',\
                                           aggfunc = ['sum', 'count', 'mean'])

#создаем сводную таблицу с отображением соотношения должников в процентах
pivot_purpose_debt = borrowers.pivot_table(index = ['purpose_lem'], values = 'debt')
pivot_purpose_debt['debt'] = pivot_purpose_debt['debt'].apply(lambda x: '{:.2%}'.format(x))
pivot_purpose_debt

,debt
purpose_lem,
автомобиль,9.36%
недвижимость,7.23%
образование,9.22%
свадьба,8.00%


После ранее проведенной лемматизации создадим сводную таблицу, где выведем ключевые цели кердита и количество клиентов: общее и с задолженностью. Вычислим процентное отношение этих величин при разных целях.  
В результате видим, что наивысший процент задолженностей имеют клиенты, обратившиеся за кредитом на покупку автомобиля и получение образования. Лучше обстоит дело с возвратом задолженностей  у клиентов, которым нужен был кредит для покупки недвижимости и организации свадьбы. Меньше всего процент задолженности у клиентов, которые взяли кредит на приобретение жилья.

### Общий вывод

Количество детей клиента влияет на факт погашения кредита в срок. Наивысший процент задолженностей имеют клиенты с одним ребенком(444 человека из 4818 или 9.22%), двумя(194 из 2055 клиентов, 9.44%) и четырьмя детьми(4 из 41 человек, 9.76%), хотя и имеющие троих детей находятся примерно на том же уровне(27 из 330 клиентов, 8.18%). Данные о имеющих пятерых детей несущественны для статистики, поскольку таких случаев слишком мало (из девяти человек, включенных в базу, ни один не имел залолженности(0.00%)).
Таким образом, меньше всего процент задолженностей у бездетных.

Семейное положение клиента влияет на факт погашения кредита в срок. Наивысший процент задолженностей имеют клиенты, находящиеся не в браке(274 человека из 2813 или 9.74%) либо живущие в гражданском браке(388 из 4177 клиентов, 9.29%). Чуть лучше обстоит дело с возвратом задолженностей  у состоящих в браке(931 человек из 12380 или 7.52%) и разведенных(85 из 1195, 7.11%). Меньше всего процент задолженности у вдовцов(63 человека из 960, 6.56%).

Уровень дохода клиента влияет на факт погашения кредита в срок. Исходя из расчетов, получается, что наивысший уровень задолженности имеют две средние категории из четырех. Для категории 2 задолженность имеют 385 человек из 4419 или 8.71% (заработок от 107798 до 142594р). Для 3 категории - 543 человека из 6343 или 8.56% (заработок от 142594 до 195549р). Чуть лучше обстоят дела с возвратом кредита у самой низшей, 1 категории (427 человек из 5381 или 7.94%, заработок 107798р и ниже). И лучше всего ситуация у 4 категории (долг имеют 386 человек из 5382 или 7.17%, уровень дохода от 195550р и выше).

Цели кредита влияют на факт его погашения в срок. Наивысший процент задолженностей имеют клиенты, обратившиеся за кредитом на покупку автомобиля(403 человека из 4315 или 9.34%) и получение образования(370 из 4022, 9.20%). Лучше обстоит дело с возвратом задолженностей  у клиентов, которым нужен был кредит для покупки недвижимости(782 клиентов из 10840, 7.21%) и организации свадьбы(186 из 2348, 7.92%). 